# **AM 207**: Homework 9 - Part 1

_ _ _ _ _

Verena Kaynig-Fittkau & Pavlos Protopapas <br>
Handed out: Wednesday, April 8, 2015<br>
Due: 11.59 P.M. Wednesday, April 15, 2015<br>

**Instructions**:

+ Upload your answers in an ipython notebook to the dropbox.

+ Your individual submissions should use the following filename: AM207_YOURNAME_HW9.ipynb

+ Your code should be in code cells as part of your ipython notebook. Do not use a different language (or format) unless you get permission from the TFs. If you use any special libraries you must include them with your code (program should run as is). 

+ If you have multiple files (e.g. you've added code files and images) create a tarball for all files in a single file and name it: AM207_YOURNAME_HW9.tar or AM207_YOURNAME_HW9.zip

+ Please remember that your solution should be a report! We would like some explanations of your ideas, and ways to approach the solution. Also please comment your code.

** Important **

As you are all busy working on your final projects, HW9 and HW10 are half problem sets. This means that each HW has only one problem to solve. There will be a dropbox for each HW, but they will be graded together. If you still have sufficient late days, you can use one late day for HW9 and one late day for HW10. 
_ _ _ _ _

In [111]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
import random
import math
import time
import seaborn as sns
#sns.set_style('white')
#sns.set_context('paper')

## Question 1: Sam let me be

[Green Eggs and Ham](http://en.wikipedia.org/wiki/Green_Eggs_and_Ham) is a best-selling and critically acclaimed children's book by Dr. Seuss, first published on August 12, 1960. As of 2001, according to Publishers Weekly, it was the fourth best-selling English-language children's book of all time.

The book is actually the result of a bet between Dr Seuss and his publisher Bennett Cerf. Cerf bet Seuss that he could not complete an entire book using only fifty different words. 

You can find the entire text of the book in the file eggs.txt. The file is conveniently formatted to not include any punctuation signs. 

### Part 1: 

Use the text of the children's book to build a simple statistical language model for the writing style of green eggs and ham. 
Your model should be a _simple Markov chain of order 1_. Use the model to generate some sentences that are longer than the standard sentences in the file. 

Note: Each line in the text file is assumed to contain exactly one sentence. 

* Filter the text to get all fifty unique words of the text
* Generate a vector for the starting probabilities of the chain (the first word of the sentence). 
* Generate a matrix with the transition probabilities of all words. 
* Add an additional row and column to your matrix to stand for the end of the sentence. Choose whatever symbol you like, I went for a '.' . The last word of a sentence should transition to this symbol and it only transitions to itself. 
* Generate and print 10 different sequences of 20 states from your model. This should correspond to 10 sentences, which might end in multiple '.', or also not end yet within the 20 states. 

Hint: It is best to convert all words to lower case. 

### Part 2:

Extend your model to a Hidden Markov Model, that can correct spelling mistakes. The true word is the hidden state $X_i$ and the observed misspelled word is the $Y_i$. Identify the correct spelling by using the Viterbi algorithm to find the most probable chain of states $X$ that caused the observations $Y$. 

You already have the starting and transition probabilities from part 1. For an HMM you also need emission probabilities. We model the emission probabilities according to a form of edit distance: 

$$p(Y_i | X_i) = \begin{cases}
                       Poisson(k, \lambda),& \text{if } length(Y_i) == length(X_i)\\
                        0,              & \text{otherwise}
\end{cases}.$$ 

Here $k = d(X_i,Y_i)$ is the number of characters that are misspelled, for example $d(Sam, Tom) = 2$. You can play around a bit with the value for $\lambda$, if you don't want to play around a value of 0.01 should work fine.  

Use the Viterbi algorithm to correct the following sentences, and print the corrected version. There is a version of the Viterbi algorithm in the lecture notes that you can use for your reference. Please adjust the code to deal with numerical underflow and add a lot of comments that show you understand what is going on. 

## part1

Use the text of the children's book to build a simple statistical language model for the writing style of green eggs and ham. 
Your model should be a _simple Markov chain of order 1_. Use the model to generate some sentences that are longer than the standard sentences in the file. 

Note: Each line in the text file is assumed to contain exactly one sentence. 

* Filter the text to get all fifty unique words of the text
* Generate a vector for the starting probabilities of the chain (the first word of the sentence). 
* Generate a matrix with the transition probabilities of all words. 
* Add an additional row and column to your matrix to stand for the end of the sentence. Choose whatever symbol you like, I went for a '.' . The last word of a sentence should transition to this symbol and it only transitions to itself. 
* Generate and print 10 different sequences of 20 states from your model. This should correspond to 10 sentences, which might end in multiple '.', or also not end yet within the 20 states. 

Hint: It is best to convert all words to lower case. 

###Filter the text to get all fifty unique words of the text

In [112]:
filename = 'eggs.txt'

with open(filename) as f:
    data = f.readlines()

# delete \n of every row
for idx, row in enumerate(data):
    data[idx] = row[:-1]

In [113]:
# use set to store every unique word
words = set()

for row in data:
    for word in row.split(' '):
        if word!='':
            words.add(word.lower())

Generate a vector for the starting probabilities of the chain (the first word of the sentence).
Generate a matrix with the transition probabilities of all words.
Add an additional row and column to your matrix to stand for the end of the sentence. Choose whatever symbol you like, I went for a '.' . The last word of a sentence should transition to this symbol and it only transitions to itself.

In [114]:
# build a map from word to index
word_to_index = {}
index_to_word = []
for idx, word in enumerate(words):
    index_to_word.append(word)
    word_to_index[word] = idx

In [115]:
# initialize start probability and transition probability
start_prob = np.zeros(51)
transit_prob = np.zeros((51,51))

for row in data:
    row_words = row.split(' ')
    for idx, word in enumerate(row_words):
        word = word.lower()
        
        # ignore ''
        if word=='':
            continue
            
        # if first word, add to start probability
        if idx==0:
            prev_idx = word_to_index[word]
            start_prob[prev_idx]+=1.
            
            
        # if it is not last word
        elif idx<len(row_words)-1:
            
            transit_prob[prev_idx][word_to_index[word]] += 1.
            prev_idx = word_to_index[word]
        
        # if it is last word
        else:
            transit_prob[prev_idx][word_to_index[word]] += 1.
            prev_idx = None
            transit_prob[word_to_index[word]][50] += 1.

# '.' goes to itself
transit_prob[50][50]=1

# normalize them to probability
transit_prob /= np.sum(transit_prob, axis = 1)[:,None]
start_prob /= np.sum(start_prob)

In [116]:
# add '.' to words
word_to_index['.']=50
index_to_word.append('.')

Generate and print 10 different sequences of 20 states from your model. This should correspond to 10 sentences, which might end in multiple '.', or also not end yet within the 20 states.

In [75]:
num_sents = 10
num_states = 20

sentences = []

for i in xrange(num_sents):
    sent = []
    for j in xrange(num_states):
        if j==0:
            prev = np.random.choice(50, p=start_prob)
            sent.append(index_to_word[prev])
        else:
            prev = np.random.choice(51, p=transit_prob[prev])
            sent.append(index_to_word[prev])
            
    sentences.append(' '.join(sent))

In [78]:
print "Generated sentences are:"
sentences

Generated sentences are:


['i will not in a train . . . . . . . . . . . . . .',
 'not could not with a fox . . . . . . . . . . . . . .',
 'i am . . . . . . . . . . . . . . . . . .',
 'and ham . . . . . . . . . . . . . . . . . .',
 'i am . . . . . . . . . . . . . . . . . .',
 'not like green eggs and i could you could not like them . . . . . . . .',
 'sam i will not like them in a boat . . . . . . . . . . .',
 'i do not would not with a goat . . . . . . . . . . . .',
 'i would not like that sam let me be . . . . . . . . . . .',
 'i will eat them sam let me be i would not on a mouse . . . . . .']

### Part 2:

Extend your model to a Hidden Markov Model, that can correct spelling mistakes. The true word is the hidden state $X_i$ and the observed misspelled word is the $Y_i$. Identify the correct spelling by using the Viterbi algorithm to find the most probable chain of states $X$ that caused the observations $Y$. 

You already have the starting and transition probabilities from part 1. For an HMM you also need emission probabilities. We model the emission probabilities according to a form of edit distance: 

$$p(Y_i | X_i) = \begin{cases}
                       Poisson(k, \lambda),& \text{if } length(Y_i) == length(X_i)\\
                        0,              & \text{otherwise}
\end{cases}.$$ 

Here $k = d(X_i,Y_i)$ is the number of characters that are misspelled, for example $d(Sam, Tom) = 2$. You can play around a bit with the value for $\lambda$, if you don't want to play around a value of 0.01 should work fine.  

Use the Viterbi algorithm to correct the following sentences, and print the corrected version. There is a version of the Viterbi algorithm in the lecture notes that you can use for your reference. Please adjust the code to deal with numerical underflow and add a lot of comments that show you understand what is going on. 

In [158]:
# emission probability
def log_emit_p(X, Y):
    if len(X)!=len(Y):
        return np.log(1e-30)
    else:
        k = len(set(X)-set(Y))
        return sp.stats.poisson.logpmf(k,lamb)

In [150]:
def print_dptable(V):
    s = "    " + " ".join(("%7d" % i) for i in range(len(V))) + "\n"
    for y in V[0]:
        s += "%.5s: " % y
        s += " ".join("%.7s" % ("%f" % v[y]) for v in V)
        s += "\n"
    print(s)

Underflow will appear because we multiply them by too many small value. In order to deal with underflow, I can use log-scale probability to implement the viterbi algorithm. However, we also need to deal with 0 probability in original scale. To deal with 0 probability, I use a small value 1e-30 to approximate 0. Therefore, I recalculate the start probability and transition probability in log-scale.

In [148]:
log_start_prob = start_prob.copy()
log_tran_prob = transit_prob.copy()

log_start_prob[log_start_prob==0]=1e-30
log_tran_prob[log_tran_prob==0]=1e-30

log_start_prob = np.log(log_start_prob)
log_tran_prob = np.log(log_tran_prob)

In [169]:
def viterbi(obs, states, log_start_p, log_trans_p, log_emit_p, print_V=False):
    """
    Input observation, states, start probability, transition probability and emission probability
    Output a tuple of probability and true path
    """
    V = [{}]
    path = {}
 
    # Initialize base cases (t == 0)
    for y, word in enumerate(states):
        V[0][word] = log_start_p[y] + log_emit_p(obs[0],word)
        path[word] = [word]
 
    # Run Viterbi for t > 0
    for t in range(1, len(obs)):
        V.append({})
        newpath = {}
 
        # iterate every state
        for y, word in enumerate(states):
            # greedily choose the partial path with maximum probability 
            (prob, state) = max((V[t-1][word0] + log_trans_p[y0][y] + log_emit_p(obs[t], word), word0) 
                                for y0, word0 in enumerate(states))
            # update to V
            V[t][word] = prob
            
            # add a new word
            newpath[word] = path[state] + [word]
 
        # add new path to the old paths
        path = newpath
    
    if print_V:
        print_dptable(V)
    
    # finally, choose the path with maximum probability
    (prob, state) = max((V[t][word], word) for y, word in enumerate(states))
    return path[state]

In [170]:
observation1 = ('nat','im','a','cor','.')
observation2 = ('yoo','lat','ma','be','.')
observation3 = ('i', 'do', 'nat', 'eet', 'tnem', 'san', 'i', 'do', 'nat', 'like', 'grean', 'egxs', 'ant', 'hom', '.')
observation4 = ('xxx','will','see','.')
observation5 = ('do','you','like','xxxxx','xxxx','and','xxx','.')
observation6 = ('x', 'xx', 'xxx', 'xxx', 'xxxx', 'xxx', 'x', 'xx', 'xxx', 'xxxx', 'xxxxx', 'xxxx', 'xxx', 'xxx', '.')

obs = [observation1, observation2, observation3, observation4, observation5, observation6]

In [172]:
corrected = viterbi(observation1, index_to_word, log_start_prob, log_tran_prob, log_emit_p, print_V=True)
print "Original:", observation1
print "Corrected:", corrected

          0       1       2       3       4
and: -7.0575 -144.75 -150.89 -97.768 -157.99
sam: -13.742 -91.039 -147.96 -97.768 -157.99
do: -74.004 -31.181 -147.36 -151.22 -157.99
house: -138.15 -143.71 -150.89 -84.165 -157.99
am: -138.15 -76.322 -148.22 -151.22 -157.99
see: -84.694 -90.345 -150.50 -97.768 -157.99
are: -78.991 -143.08 -150.89 -92.064 -157.99
a: -72.618 -142.71 -13.073 -151.22 -93.529
in: -73.311 -12.741 -148.50 -151.22 -157.99
mouse: -138.15 -143.71 -150.89 -84.165 -157.99
boat: -138.15 -144.69 -150.89 -85.146 -157.99
if: -74.004 -80.290 -150.89 -151.22 -157.99
will: -138.15 -80.094 -148.03 -151.22 -157.99
ham: -78.991 -77.010 -150.89 -97.768 -157.99
would: -71.807 -79.401 -148.29 -151.22 -157.99
there: -138.15 -79.313 -150.89 -151.22 -157.99
fox: -84.694 -143.84 -150.89 -25.135 -157.99
.: -138.15 -82.819 -39.380 -86.245 -19.846
green: -138.15 -81.104 -147.20 -151.22 -157.99
you: -18.465 -78.619 -103.84 -92.064 -157.99
goat: -138.15 -144.40 -150.89 -84.858 -157.99
be: -1

From the plot of V above, we can see that we fix the underflow problem.

In [165]:
lambs = [0.005, 0.01, 0.02, 0.03, 0.05, 0.1]
for value in lambs:
    lamb = value
    print "When Lambda is %.3f. The corrected sentences are:"%lamb
    for ob in obs:
        print viterbi(ob, index_to_word, log_start_prob, log_tran_prob, log_emit_p)
    print

When Lambda is 0.005. The corrected sentences are:
['not', 'in', 'a', 'car', '.']
['you', 'let', 'me', 'be', '.']
['i', 'do', 'not', 'eat', 'them', 'sam', 'i', 'do', 'not', 'like', 'green', 'eggs', 'and', 'ham', '.']
['you', 'will', 'see', '.']
['do', 'you', 'like', 'green', 'eggs', 'and', 'ham', '.']
['i', 'do', 'not', 'eat', 'them', 'sam', 'i', 'do', 'not', 'like', 'green', 'eggs', 'and', 'ham', '.']

When Lambda is 0.010. The corrected sentences are:
['not', 'in', 'a', 'car', '.']
['you', 'let', 'me', 'be', '.']
['i', 'do', 'not', 'eat', 'them', 'sam', 'i', 'do', 'not', 'like', 'green', 'eggs', 'and', 'ham', '.']
['you', 'will', 'see', '.']
['do', 'you', 'like', 'green', 'eggs', 'and', 'ham', '.']
['i', 'do', 'not', 'eat', 'them', 'sam', 'i', 'do', 'not', 'like', 'green', 'eggs', 'and', 'ham', '.']

When Lambda is 0.020. The corrected sentences are:
['not', 'in', 'a', 'car', '.']
['you', 'let', 'me', 'be', '.']
['i', 'do', 'not', 'eat', 'them', 'sam', 'i', 'do', 'not', 'like', 'gree

I think $\lambda $ should be small because when $\lambda$ is large, the emission probability will dominate the partial path probability. The effect of transition probability is much less and this algorithm will always choose the most similar word. If we have a correct $\lambda$, we can balance the emission probability and transition probability and find the most possible path. Therefore, according to the output above, I choose my $\lambda$ as 0.01.

In [168]:
lamb = 0.01
print "Our final corrected sentences are:"
for ob in obs:
    print ' '.join(viterbi(ob, index_to_word, log_start_prob, log_tran_prob, log_emit_p))

Our final corrected sentences are:
not in a car .
you let me be .
i do not eat them sam i do not like green eggs and ham .
you will see .
do you like green eggs and ham .
i do not eat them sam i do not like green eggs and ham .


### Optional

If you would like to extend your knowledge of HMM models with this example, feel free to build models on the level of single characters, and build models of higher order. You could also try different texts, for example Shakespeare's Sonnets. 